# Quick exploration of geovec, a pre-trained word embedding in gloVe format

#### Data file of the pretrained embedding is downloaded from these links:

download site & information https://osf.io/4uyeq/wiki/home/

h5 file = https://osf.io/4uyeq/

Medium article by authors: https://towardsdatascience.com/geovec-word-embeddings-for-geosciences-ac1e1e854e19

In [1]:
#conda install -c anaconda scikit-learn 
# !pip install h5py
#!conda install -c anaconda h5py 

In [2]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

In [3]:
import json
import h5py
import numpy as np

##### glove python is a dependency mentioend by the authors
##### DON'T DO THIS AS IT HAS AlL SORTS OF PROBLEMS ON MACOS AND WINDOWS

#!pip install glove_python

### Instead of using glove-python, we can just open the file this way

In [4]:
with h5py.File('geovec_300d_v1.h5', 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)

In [5]:
#### number of words with vector embeddings
print("number of words with vector embeddings",len(v))
#### Length of vectors
print("number of vectors in each embedding",len(v[1]))

number of words with vector embeddings 411845
number of vectors in each embedding 300


In [6]:
print("number of words total",len(dct))
print("and what each word in dictionary looks like: ",list(dct.keys())[0:1][0],":",dct[list(dct.keys())[0:1][0]])

number of words total 411845
and what each word in dictionary looks like:  djth : 225512


In [7]:
print("embedding for acrodermatitis",v[dct[list(dct.keys())[0:1][0]]])

embedding for acrodermatitis [-6.3330486e-02 -1.2821836e-02  5.5139054e-02  5.1858775e-02
  3.7441850e-02  4.3557122e-02 -6.2944531e-02  1.6824855e-02
  5.4459341e-02  2.1199655e-02 -5.9035216e-03 -2.6880730e-02
 -4.2180978e-02  7.6353051e-02  1.4817851e-02  5.6957360e-02
  3.7775230e-02  6.3957781e-02  5.1477615e-02 -6.6322368e-04
 -5.6233235e-02 -3.5311762e-02 -6.5173432e-02  2.9121403e-02
  4.6766844e-02 -4.6924517e-02 -3.3553474e-02  5.9291858e-02
 -5.7618013e-03  3.5043120e-02  3.6559843e-02  7.1166418e-02
 -1.7607285e-02 -4.4483110e-02  8.3809318e-03  1.5739698e-02
 -4.0424205e-02  2.7711699e-02 -9.8132454e-03  6.0936373e-02
 -5.5786024e-04 -4.1178890e-02  2.4220731e-02 -7.9250962e-02
 -6.7355759e-02  9.9167023e-03  1.6234049e-03  6.7060828e-02
  5.2843086e-02  7.3198401e-03 -7.5139076e-02 -3.0536870e-02
 -7.6598683e-03  5.8626261e-02  4.9598049e-02  6.4568967e-02
 -1.5757466e-02 -8.5722706e-05  3.6245093e-02 -4.2100079e-02
 -2.4650073e-02  2.8702517e-03  1.0773007e-02 -1.0743470

In [8]:
list(dct.keys())[0:10]

['djth',
 'pizarras',
 'prefossette',
 'lophine',
 'conifundus',
 'mgpr',
 'spelunker',
 'delicatulus',
 'hbsrpp',
 'tkp']

In [9]:
type(v)

numpy.ndarray

In [10]:
#v

### We'll combine the two files into a single dictionary to make some later things a bit easier. It isn't saved in thi format to save space according to the authors who trained it

In [11]:
def replaceIntsWithWords(vectors,dictionary):
    words = list(dct.keys())
    embeddings_dict = {}
    for word in words:
        embeddings_dict[word] = np.asarray(v[dictionary[word]][:],"float32")
    return embeddings_dict

In [12]:
new_vector_embedding = replaceIntsWithWords(v,dct)

In [13]:
new_vector_embedding["sandstone"]

array([ 0.03063519, -0.41190132, -0.01748249, -0.03562951, -0.44922313,
       -0.23300318,  0.22224723, -0.07885347,  0.02758265, -0.07660752,
       -0.39882764,  0.19357805, -0.03862172, -0.16392523, -0.08809105,
       -0.15323128, -0.17489772,  0.14711891,  0.10426124, -0.02329986,
        0.09977868,  0.23435533,  0.3037015 , -0.6652086 , -0.1779219 ,
        0.16997509, -0.0749488 , -0.4623773 , -0.18799299, -0.04015233,
       -0.1230348 , -0.324141  ,  0.32538635,  0.27751258, -0.04844321,
        0.04106978,  0.3250498 ,  0.4131149 ,  0.41197386,  0.05734786,
       -0.28493047,  0.05330373, -0.30320197, -0.6228066 ,  0.17232999,
        0.13726279,  0.02862607,  0.00987866, -0.22104132, -0.27486506,
        0.3875012 ,  0.05712619,  0.30534878, -0.17705044, -0.19142899,
       -0.1692242 , -0.3759296 ,  0.35963646, -0.02505949,  0.05952422,
        0.33059987,  0.18033917,  0.05865888,  0.1908917 ,  0.03191795,
       -0.3452011 , -0.4242747 ,  0.02242155, -0.08702113, -0.36

### Now let's play around a bit to find out what words are close to others in the embedding space

In [14]:
def find_closest_embeddings(embeddings_dict,embedding_vector):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding_vector))

In [15]:
len(new_vector_embedding.keys())

411845

In [16]:
spatial.distance.euclidean(new_vector_embedding['sandstone'], new_vector_embedding['granite'])

4.911217212677002

In [17]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["sandstone"])[0:10]

['sandstone',
 'siltstone',
 'mudstone',
 'arkosic',
 'conglomerate',
 'rothbach',
 'gröden',
 'sonta',
 'mudstones',
 'interbedded']

In [18]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["granite"])[0:10]

['granite',
 'granitic',
 'ebat',
 'tuaní',
 'kibe',
 'lavadores',
 'baerzhe',
 'daguzhai',
 'cooyerdoo',
 'qôrqut']

In [19]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["crossbedding"])[0:20]

['crossbedding',
 'lumi',
 'unrein',
 'superface',
 'boundinaged',
 'staretsky',
 'essikado',
 'gerlarch',
 'doucette',
 'szenleng',
 'rhune',
 'shihui',
 'gackowa',
 'lenticularity',
 'tenappera',
 'karlshafen',
 'håkjerringdjupet',
 'anorbital',
 'eludicate',
 'torom']

### Looks like this is a lot of non-English words?!!

### Let's try subtraction of pure vectors

In [20]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["stratum"] - new_vector_embedding["fault"]
)[:25])

['sagyeri', 'pregrowth', 'barzaloza', 'translatent', 'syntransform', 'dothan', 'superincumbent', 'corniola', 'adoudou', 'chingmei', 'březno', 'saberian', 'kulm', 'trentinara', 'flamy', 'chulca', 'setlaole', 'emborozu', 'ikpe', 'guxu', 'aib', 'sarsbukta', 'huilu', 'nunda', 'lecho']


In [21]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["marble"] - new_vector_embedding["heat"]
)[:25])

['marble', 'sedef', 'pentelic', 'supren', 'macael', 'yeosan', 'calanchiole', 'pentelikon', 'kemalpasa', 'dionysos', 'borba', 'alabashka', 'iscehisar', 'maniçoba', 'hekkelstrand', 'benou', 'calcsilicates', 'evenes', 'harsin', 'montepuez', 'ayhan', 'nanshu', 'sanchuan', 'belevi', 'bettadabidu']


In [22]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["boulder"] - new_vector_embedding["clay"]
)[:25])

['intratill', 'satitoa', 'boulder', 'montoz', 'laxe', 'kintradwell', 'unclimbed', 'settai', 'norber', 'alveolized', 'intertill', 'cipit', 'preexposure', 'kudaka', 'moutonnee', 'ventifacted', 'navabandar', 'sturgis', 'taufererberg', 'pineo', 'preexposed', 'mathu', 'runswick', 'spelonk', 'encierro']


### A blunt approach doesn't work so well

## Let's try to use tensorflow's embedding projector

In [23]:
!pip install torch

In [24]:
!pip install fasttext

https://stackoverflow.com/questions/41258391/tensorboard-embedding-example/41262360#41262360

In [25]:
import os
import tensorflow as tf
import numpy as np
#import fasttext
from tensorflow.contrib.tensorboard.plugins import projector



In [26]:
tf

<module 'tensorflow' from '/Users/justingosses/anaconda/lib/python3.5/site-packages/tensorflow/__init__.py'>

In [27]:

# # load model
# word2vec = fasttext.load_model('wiki.en.bin')
# # load model
# #word2vec = fasttext.load_model('wiki.en.bin')

In [28]:
len(v[0])

300

In [29]:
len(dct)

411845

In [30]:
list(dct.keys())

['djth',
 'pizarras',
 'prefossette',
 'lophine',
 'conifundus',
 'mgpr',
 'spelunker',
 'delicatulus',
 'hbsrpp',
 'tkp',
 'scalit',
 'gioia',
 'motovsky',
 'norhopanes',
 'sertãozinho',
 'semlikiichthys',
 'fbf',
 'cheilocypridea',
 'phlogoiderhynchus',
 'ixi',
 'pararaia',
 'tbux',
 'kuzuryugawa',
 'subaereally',
 'spouted',
 'chemico',
 'sörlin',
 'vqm',
 'isuela',
 'chibaite',
 'oew',
 'ebsb',
 'genomically',
 'lipase',
 'herre',
 'barkan',
 'supergiant',
 'hyperstenichnus',
 'sicca',
 'maghémite',
 'constrain',
 'imai',
 'yuanjiagou',
 'geofyzika',
 'ibadan',
 'wulian',
 'pelodytes',
 'bgdgt',
 'partha',
 'pheromonal',
 'subcentimetric',
 'dewt',
 'brevimastigomonas',
 'citrinum',
 'intriguingly',
 'insister',
 'desulfohalobium',
 'peanut',
 'brezno',
 'briani',
 'monothiocarbonate',
 'eqution',
 'trödje',
 'lebowa',
 'microbiofacies',
 'xanthation',
 'excdo',
 'castellan',
 'porridge',
 'portaeferreae',
 'methylarsenical',
 'biosis',
 'söderbaum',
 'chubbuck',
 'backpart',
 'aps

In [31]:
v

array([[ 0.6575543 , -0.10153764, -0.2825202 , ...,  0.64620304,
         0.3083481 , -0.46784   ],
       [ 0.06665467,  0.11548886,  0.22080564, ..., -0.08355215,
         0.00378662, -0.13758856],
       [ 0.32676426,  0.26151726, -0.33332917, ...,  0.15112405,
         0.00892717, -0.19385871],
       ...,
       [ 0.02525451, -0.00096976,  0.04692964, ..., -0.05422716,
        -0.02048381,  0.01153759],
       [ 0.09515277,  0.09372371,  0.09677262, ..., -0.10224963,
        -0.06350729, -0.05045411],
       [-0.02760863,  0.0040131 ,  0.03059224, ..., -0.04461955,
        -0.03546656, -0.00330813]], dtype=float32)

In [32]:
# # create a list of vectors
embedding = np.empty((len(dct), len(v[0])), dtype=np.float32)
i = 0
words = list(dct.keys())
for word in words:
    embedding[i] = v[dct[word]]
    i+=1



In [33]:
embedding

array([[-0.06333049, -0.01282184,  0.05513905, ..., -0.0413099 ,
        -0.04549453,  0.02310135],
       [ 0.02773892, -0.02880791, -0.03011626, ...,  0.01685739,
        -0.05396912,  0.08621652],
       [ 0.13176115,  0.01681024,  0.23243658, ...,  0.02557935,
        -0.1608475 ,  0.05357835],
       ...,
       [-0.04044866, -0.00364208, -0.02147772, ...,  0.03959141,
         0.00173467,  0.00903813],
       [-0.03686953,  0.02338972,  0.12521453, ..., -0.14274253,
        -0.14653201,  0.12951678],
       [ 0.05561383, -0.17315626, -0.06620983, ..., -0.01180434,
        -0.00871795, -0.09236129]], dtype=float32)

In [34]:
words

['djth',
 'pizarras',
 'prefossette',
 'lophine',
 'conifundus',
 'mgpr',
 'spelunker',
 'delicatulus',
 'hbsrpp',
 'tkp',
 'scalit',
 'gioia',
 'motovsky',
 'norhopanes',
 'sertãozinho',
 'semlikiichthys',
 'fbf',
 'cheilocypridea',
 'phlogoiderhynchus',
 'ixi',
 'pararaia',
 'tbux',
 'kuzuryugawa',
 'subaereally',
 'spouted',
 'chemico',
 'sörlin',
 'vqm',
 'isuela',
 'chibaite',
 'oew',
 'ebsb',
 'genomically',
 'lipase',
 'herre',
 'barkan',
 'supergiant',
 'hyperstenichnus',
 'sicca',
 'maghémite',
 'constrain',
 'imai',
 'yuanjiagou',
 'geofyzika',
 'ibadan',
 'wulian',
 'pelodytes',
 'bgdgt',
 'partha',
 'pheromonal',
 'subcentimetric',
 'dewt',
 'brevimastigomonas',
 'citrinum',
 'intriguingly',
 'insister',
 'desulfohalobium',
 'peanut',
 'brezno',
 'briani',
 'monothiocarbonate',
 'eqution',
 'trödje',
 'lebowa',
 'microbiofacies',
 'xanthation',
 'excdo',
 'castellan',
 'porridge',
 'portaeferreae',
 'methylarsenical',
 'biosis',
 'söderbaum',
 'chubbuck',
 'backpart',
 'aps

In [35]:
# # create a list of vectors
# embedding = np.empty((len(word2vec.words), word2vec.dim), dtype=np.float32)
# for i, word in enumerate(word2vec.words):
#     embedding[i] = word2vec[word]

In [36]:
!mkdir 'log'
!cd log
!touch 'metadata.tsv'
!cd ..

mkdir: log: File exists


In [40]:
# setup a TensorFlow session
tf.reset_default_graph()
sess = tf.InteractiveSession()
X = tf.Variable([0.0], name='embedding')
place = tf.placeholder(tf.float32, shape=embedding.shape)
set_x = tf.assign(X, place, validate_shape=False)
sess.run(tf.global_variables_initializer())
sess.run(set_x, feed_dict={place: embedding})

# write labels
with open('log/metadata.tsv', 'w') as f:
#     for word in word2vec.words:
    for word in words:
        f.write(word + '\n')

# create a TensorFlow summary writer
summary_writer = tf.summary.FileWriter('log', sess.graph)
config = projector.ProjectorConfig()
embedding_conf = config.embeddings.add()
embedding_conf.tensor_name = 'embedding:0'
embedding_conf.metadata_path = os.path.join('log', 'metadata.tsv')
projector.visualize_embeddings(summary_writer, config)

# save the model
saver = tf.train.Saver()
saver.save(sess, os.path.join('log', "model.ckpt"))

KeyboardInterrupt: 

In [ ]:
!tensorboard --logdir=log

In [39]:
#!pip install mkl

In [41]:
import io


In [ ]:

encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()


In [46]:
!cd log
!touch 'log/vecs.tsv'
!cd ..

In [50]:

out_v = io.open('log/vecs.tsv', 'w', encoding='utf-8')

for n,weights in enumerate(embedding):
    vec = v[n]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")